# **XGBoost**


In [82]:
# Enviroment
isColab = False
colab_dir = "/gdrive/My Drive/Colab Notebooks/[2025-2026] AN2DL/AN2DL-challenge-2"

isKaggle = False
isWsl = True

# Set seed for reproducibility
SEED = 42

In [83]:
# Loader parameters
BATCH_SIZE = 32

NORMALIZATION_MEAN = [0.485, 0.456, 0.406]
NORMALIZATION_STD = [0.229, 0.224, 0.225]

IMG_RESIZE = (300, 300)
INPUT_SHAPE = (3, *IMG_RESIZE)

MASK_THRESHOLD = 0.01

In [84]:
EXPERIMENT_NAME = "densenet121_300p_v2"
NET_NAME = "densenet121"

# Fine tuning parameters
FT_DROPOUT_RATE = 0.45

K_FOLD_LIMIT = 3
K_FOLD_MAX_VALUE = 5

submission_path = f"submission/{EXPERIMENT_NAME}_submission_xgb.csv"

## **Loading Enviroment**


In [85]:
import os

# Directory di default
current_dir = os.getcwd()   

if isColab:
    from google.colab import drive # type: ignore
    drive.mount("/gdrive")
    current_dir = colab_dir
    print("In esecuzione su Colab. Google Drive montato.")
    %cd $current_dir
elif isKaggle:
    kaggle_work_dir = "/kaggle/working/AN2DL-challenge-2"
    os.makedirs(kaggle_work_dir, exist_ok=True)
    current_dir = kaggle_work_dir
    print("In esecuzione su Kaggle. Directory di lavoro impostata.")
    os.chdir(current_dir)
elif isWsl:
    local_pref = r"/mnt/g/Il mio Drive/Colab Notebooks/[2025-2026] AN2DL/AN2DL-challenge-2"
    current_dir = local_pref if os.path.isdir(local_pref) else os.getcwd()
    print(f"Esecuzione su WSL. Directory corrente impostata a: {current_dir}")
    os.chdir(current_dir)
else:
    print("Esecuzione locale. Salto mount Google Drive.")
    local_pref = r"G:\Il mio Drive\Colab Notebooks\[2025-2026] AN2DL\AN2DL-challenge-2"
    current_dir = local_pref if os.path.isdir(local_pref) else os.getcwd()
    print(f"Directory corrente impostata a: {current_dir}")
    os.chdir(current_dir)

print(f"Changed directory to: {current_dir}")

# Define absolute paths
dataset_dir = os.path.join(current_dir, "dataset")
train_set_dir = os.path.join(dataset_dir, "train_data")
test_set_dir = os.path.join(dataset_dir, "test_data")
label_file = os.path.join(dataset_dir, "train_labels.csv")

print(f"Dataset directory: {dataset_dir}")
print(f"Train set directory: {train_set_dir}")
print(f"Test set directory: {test_set_dir}")
print(f"Label file: {label_file}")

Esecuzione su WSL. Directory corrente impostata a: /mnt/g/Il mio Drive/Colab Notebooks/[2025-2026] AN2DL/AN2DL-challenge-2
Changed directory to: /mnt/g/Il mio Drive/Colab Notebooks/[2025-2026] AN2DL/AN2DL-challenge-2
Dataset directory: /mnt/g/Il mio Drive/Colab Notebooks/[2025-2026] AN2DL/AN2DL-challenge-2/dataset
Train set directory: /mnt/g/Il mio Drive/Colab Notebooks/[2025-2026] AN2DL/AN2DL-challenge-2/dataset/train_data
Test set directory: /mnt/g/Il mio Drive/Colab Notebooks/[2025-2026] AN2DL/AN2DL-challenge-2/dataset/test_data
Label file: /mnt/g/Il mio Drive/Colab Notebooks/[2025-2026] AN2DL/AN2DL-challenge-2/dataset/train_labels.csv


## **Import Libraries**


In [ ]:
# Set environment variables before importing modules
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(SEED)
random.seed(SEED)

# Import PyTorch
import torch
torch.manual_seed(SEED)
from torch import nn
import torchvision
from torch.utils.data import DataLoader


# Configurazione di TensorBoard e directory
logs_dir = "tensorboard"
if isColab or isKaggle:
    !pkill -f tensorboard 
    !mkdir -p models
    print("Killed existing TensorBoard instances and created models directory.") 

os.makedirs("models", exist_ok=True)  


if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

# Import other libraries
import shutil
from itertools import product
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

PyTorch version: 2.9.1+cu130
Device: cuda


### **Preparing Dataset for colab**


In [87]:
if isColab:
    drive_dataset_dir = os.path.join(current_dir, "dataset")
    local_dataset_dir = "/content/dataset"

    if not os.path.exists(local_dataset_dir):
        print(f"Copying dataset from {drive_dataset_dir} to {local_dataset_dir}...")
        try:
            shutil.copytree(drive_dataset_dir, local_dataset_dir)
            print("Copy complete.")
        except Exception as e:
            print(f"Error copying dataset: {e}")
            print("Falling back to Drive dataset (slow).")
            # If copy fails, we stick to the original dataset_dir (which might need cleaning too if it was used directly)
            dataset_dir = drive_dataset_dir
    else:
        print("Dataset already copied to local runtime.")

    # If copy succeeded (or already existed), use local path
    if os.path.exists(local_dataset_dir):
        dataset_dir = local_dataset_dir

## ⏳ **Data Loading**


### **Definitions**


In [88]:
SAMPLES_TO_IGNORE = [
    "img_0001.png",
    "img_0005.png",
    "img_0008.png",
    "img_0012.png",
    "img_0018.png",
    "img_0020.png",
    "img_0022.png",
    "img_0027.png",
    "img_0028.png",
    "img_0036.png",
    "img_0044.png",
    "img_0047.png",
    "img_0048.png",
    "img_0052.png",
    "img_0062.png",
    "img_0078.png",
    "img_0085.png",
    "img_0090.png",
    "img_0094.png",
    "img_0095.png",
    "img_0126.png",
    "img_0129.png",
    "img_0130.png",
    "img_0133.png",
    "img_0136.png",
    "img_0138.png",
    "img_0148.png",
    "img_0150.png",
    "img_0155.png",
    "img_0159.png",
    "img_0161.png",
    "img_0175.png",
    "img_0178.png",
    "img_0179.png",
    "img_0180.png",
    "img_0184.png",
    "img_0187.png",
    "img_0189.png",
    "img_0193.png",
    "img_0196.png",
    "img_0222.png",
    "img_0251.png",
    "img_0254.png",
    "img_0263.png",
    "img_0268.png",
    "img_0286.png",
    "img_0293.png",
    "img_0313.png",
    "img_0319.png",
    "img_0333.png",
    "img_0342.png",
    "img_0344.png",
    "img_0346.png",
    "img_0355.png",
    "img_0368.png",
    "img_0371.png",
    "img_0376.png",
    "img_0380.png",
    "img_0390.png",
    "img_0393.png",
    "img_0407.png",
    "img_0410.png",
    "img_0415.png",
    "img_0424.png",
    "img_0443.png",
    "img_0453.png",
    "img_0459.png",
    "img_0463.png",
    "img_0486.png",
    "img_0497.png",
    "img_0498.png",
    "img_0499.png",
    "img_0509.png",
    "img_0521.png",
    "img_0530.png",
    "img_0531.png",
    "img_0533.png",
    "img_0537.png",
    "img_0540.png",
    "img_0544.png",
    "img_0547.png",
    "img_0557.png",
    "img_0558.png",
    "img_0560.png",
    "img_0565.png",
    "img_0567.png",
    "img_0572.png",
    "img_0578.png",
    "img_0580.png",
    "img_0586.png",
    "img_0602.png",
    "img_0603.png",
    "img_0607.png",
    "img_0609.png",
    "img_0614.png",
    "img_0620.png",
    "img_0623.png",
    "img_0629.png",
    "img_0635.png",
    "img_0639.png",
    "img_0643.png",
    "img_0644.png",
    "img_0645.png",
    "img_0646.png",
    "img_0656.png",
    "img_0657.png",
    "img_0658.png",
    "img_0670.png",
    "img_0673.png",
    "img_0675.png",
]

In [89]:
# Load the full dataframe
full_df = pd.read_csv(label_file)

# Remove cursed images
full_df = full_df[~full_df["sample_index"].isin(SAMPLES_TO_IGNORE)].reset_index(
    drop=True
)

# Label mapping
class_names = sorted(full_df["label"].unique())
label_to_index = {name: idx for idx, name in enumerate(class_names)}
full_df["label_index"] = full_df["label"].map(label_to_index)
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")

Number of classes: 4


In [90]:
def make_loader(ds, batch_size, shuffle, drop_last=False):
    """Create a PyTorch DataLoader with optimized settings."""
    cpu_cores = os.cpu_count() or 2
    num_workers = max(2, min(6, cpu_cores))

    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
        pin_memory=True,
        pin_memory_device="cuda" if torch.cuda.is_available() else "",
        prefetch_factor=4,
        persistent_workers=isWsl,
    )

In [91]:
import numpy as np


class MacenkoNormalizer:
    """
    Normalizes H&E stained images to a reference appearance using the Macenko method.
    """

    def __init__(self):
        # Target reference values (standard for CPath)
        # These are pre-computed means/stds from a "perfect" slide
        self.HERef = np.array([[0.5626, 0.2159], [0.7201, 0.8012], [0.4062, 0.5581]])
        self.maxCRef = np.array([1.9705, 1.0308])

    def __call__(self, img_arr, Io=240, alpha=1, beta=0.15):
        """
        img_arr: RGB numpy array (H, W, 3)
        Returns: Normalized numpy array
        """
        # 1. Convert to Optical Density (OD)
        h, w, c = img_arr.shape
        img_arr = img_arr.reshape((-1, 3))

        # Avoid division by zero
        OD = -np.log((img_arr.astype(np.float64) + 1) / Io)

        # 2. Remove transparent pixels
        ODhat = OD[~np.any(OD < beta, axis=1)]
        if ODhat.shape[0] < 10:  # Safety check for empty patches
            return img_arr.reshape(h, w, c).copy()

        # 3. Compute SVD
        _, eigvecs = np.linalg.eigh(np.cov(ODhat.T))

        # 4. Project on the plane spanned by the eigenvectors corresponding to the two largest eigenvalues
        That = ODhat.dot(eigvecs[:, -2:])

        # 5. Find robust extremes (1st and 99th percentiles)
        phi = np.arctan2(That[:, 1], That[:, 0])
        minPhi = np.percentile(phi, alpha)
        maxPhi = np.percentile(phi, 100 - alpha)

        vMin = eigvecs[:, -2:].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
        vMax = eigvecs[:, -2:].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)

        # 6. Heuristic to ensure H is first vector, E is second
        if vMin[0] > vMax[0]:
            HE = np.array((vMin[:, 0], vMax[:, 0])).T
        else:
            HE = np.array((vMax[:, 0], vMin[:, 0])).T

        # 7. Rows correspond to channels (RGB), columns to H&E stains
        Y = np.reshape(OD, (-1, 3)).T

        # Determine concentrations of the individual stains
        C = np.linalg.lstsq(HE, Y, rcond=None)[0]

        # 8. Normalize concentrations
        maxC = np.percentile(C, 99, axis=1)
        tmp = np.divide(maxC, self.maxCRef)
        C2 = np.divide(C, tmp[:, np.newaxis])

        # 9. Reconstruct the image
        Inorm = np.multiply(Io, np.exp(-self.HERef.dot(C2)))
        Inorm[Inorm > 255] = 254
        Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)

        return Inorm

In [ ]:
from PIL import Image, ImageOps
from torch.utils.data import Dataset
from tqdm.notebook import tqdm


class MaskedGridTileDataset(Dataset):
    """
    A Dataset class that performs Grid Tiling over the tissue masks.
    It extracts patches based on a sliding window, keeping only those
    that contain sufficient biological tissue.
    """

    def __init__(
        self,
        dataframe,
        img_dir,
        transforms=None,
        target_size=(300, 300),
        mask_threshold=0.05,  # Keep patch if at least 5% is tissue
        overlap_ratio=0.0,  # 0.0 = distinct tiles; 0.5 = 50% overlap
        normalize=True,
        debug_max=None,
    ):
        self.samples = []
        self.transforms = transforms
        self.img_dir = img_dir
        self.target_size = target_size
        self.mask_threshold = mask_threshold
        self.overlap_ratio = overlap_ratio
        self.normalizer = MacenkoNormalizer() if normalize else None
        self.dropped_slides = 0

        # Determine if we are in inference mode (no labels)
        if dataframe is None or "label_index" not in dataframe.columns:
            # We are in inference mode
            if dataframe is None:
                img_names = sorted(
                    [f for f in os.listdir(img_dir) if f.startswith("img_")]
                )
            else:
                img_names = dataframe["sample_index"].tolist()
            iterator = zip(img_names, [-1] * len(img_names))
            total_items = len(img_names)
        else:
            iterator = zip(dataframe["sample_index"], dataframe["label_index"])
            total_items = len(dataframe)

        print(
            f"Processing {total_items} slides with Grid Tiling (Thr={mask_threshold}, Overlap={overlap_ratio})..."
        )

        count = 0
        for img_name, label in tqdm(iterator, total=total_items):
            if debug_max and count >= debug_max:
                break
            self._process_and_extract(img_name, label)
            count += 1

        print(f"Extraction complete. Total patches: {len(self.samples)}")
        if self.dropped_slides > 0:
            print(
                f"⚠️ Warning: {self.dropped_slides} slides were completely empty/corrupt."
            )

    def _process_and_extract(self, img_name, label):
        img_path = os.path.join(self.img_dir, img_name)
        mask_path = os.path.join(self.img_dir, img_name.replace("img_", "mask_"))

        try:
            image = Image.open(img_path).convert("RGB")
            mask = Image.open(mask_path).convert("L")
        except Exception as e:
            print(f"Warning: Could not load {img_name}: {e}")
            self.dropped_slides += 1
            return

        img_w, img_h = image.size
        tile_h, tile_w = self.target_size

        # Calculate stride based on overlap
        stride_h = int(tile_h * (1 - self.overlap_ratio))
        stride_w = int(tile_w * (1 - self.overlap_ratio))

        # Ensure stride is at least 1 pixel to prevent infinite loops
        stride_h = max(1, stride_h)
        stride_w = max(1, stride_w)

        mask_arr = np.array(mask)
        # Binarize mask (assuming any non-zero value is tissue)
        binary_mask = (mask_arr > 0).astype(np.uint8)

        # Iterate via sliding window
        # We allow y and x to go slightly out of bounds to catch edges, handled by padding later
        for y in range(0, img_h, stride_h):
            for x in range(0, img_w, stride_w):

                # Define current window coordinates
                x2 = x + tile_w
                y2 = y + tile_h

                # --- 1. MASK CHECK (Fast filtering) ---
                # Clip coordinates to image bounds for the mask check
                mx1, my1 = x, y
                mx2, my2 = min(x2, img_w), min(y2, img_h)

                # Extract the mask patch corresponding to this tile
                mask_patch = binary_mask[my1:my2, mx1:mx2]

                # If the patch is empty or purely padding, skip
                if mask_patch.size == 0:
                    continue

                # Calculate tissue percentage
                tissue_pixels = np.count_nonzero(mask_patch)
                total_pixels = tile_w * tile_h  # Use theoretical size

                # If we are on the edge, the actual mask_patch might be smaller,
                # but we normalize by the Target Tile Size to penalize mostly-empty edge crops.
                tissue_coverage = tissue_pixels / total_pixels

                if tissue_coverage < self.mask_threshold:
                    continue

                # --- 2. IMAGE EXTRACTION (Only if mask check passed) ---
                # Handle Edge Padding
                # If x2 > img_w or y2 > img_h, we need to crop what we can and pad the rest

                # Actual crop coordinates within image
                crop_x1, crop_y1 = x, y
                crop_x2, crop_y2 = min(x2, img_w), min(y2, img_h)

                patch_crop = image.crop((crop_x1, crop_y1, crop_x2, crop_y2))

                # Calculate padding needed
                pad_right = max(0, x2 - img_w)
                pad_bottom = max(0, y2 - img_h)

                if pad_right > 0 or pad_bottom > 0:
                    # Pad with white (255) for histology background
                    patch = ImageOps.expand(
                        patch_crop, border=(0, 0, pad_right, pad_bottom), fill=255
                    )
                else:
                    patch = patch_crop

                # Final safeguard on size
                if patch.size != self.target_size:
                    patch = patch.resize(self.target_size, Image.BICUBIC)  # type: ignore

                patch_arr = np.array(patch)

                if self.normalizer:
                    try:
                        # Macenko needs robust pixel data. If patch is mostly white/padding,
                        # it might fail or look weird. We only run it if we have tissue.
                        patch_arr = self.normalizer(patch_arr)
                    except Exception as e:
                        # Fallback if SVD fails on weird patches
                        print(
                            f"Normalization failed on patch from {img_name} at ({x},{y}): {e}"
                        )
                        pass

                # Store in RAM
                self.samples.append(
                    {"patch": patch_arr, "label": label, "parent": img_name}
                )

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        item = self.samples[idx]
        img = Image.fromarray(item["patch"])
        label = item["label"]

        if self.transforms:
            img = self.transforms(img)

        return img, label, item["parent"]

In [93]:
from torchvision.transforms import v2 as transforms


def compute_dataset_stats(
    dataset_class,
    dataframe,
    img_dir,
    target_size=IMG_RESIZE,
    mask_threshold=0.05,
    overlap_ratio=0.0,
    normalize=False,
):
    """
    Computes channel-wise Mean and Std on the dataset without any normalization applied.
    """
    print("Computing dataset Mean and Std (this may take a moment)...")

    # define a simple transform that only converts to tensor
    basic_transforms = transforms.Compose(
        [transforms.Resize(IMG_RESIZE), transforms.ToTensor()]
    )

    # Instantiate dataset temporarily
    temp_ds = dataset_class(
        dataframe,
        img_dir,
        transforms=basic_transforms,
        target_size=target_size,
        mask_threshold=mask_threshold,
        overlap_ratio=overlap_ratio,
        normalize=normalize,
    )
    loader = make_loader(temp_ds, batch_size=BATCH_SIZE, shuffle=False)

    mean = 0.0
    std = 0.0
    nb_samples = 0.0

    for data, _, _ in tqdm(loader):
        batch_samples = data.size(0)
        # Flatten H and W to calculate stats per channel
        data = data.view(batch_samples, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
        nb_samples += batch_samples

    mean /= nb_samples
    std /= nb_samples

    print(f"\nDONE. Copy these values into your config:")
    print(f"NEW_MEAN = {mean.tolist()}")  # type: ignore
    print(f"NEW_STD = {std.tolist()}")  # type: ignore
    return mean.tolist(), std.tolist()  # type: ignore

In [ ]:
print("Calculating stats on Training Data...")

# We use the class we just defined
custom_mean, custom_std = compute_dataset_stats(
    dataset_class=MaskedGridTileDataset,
    dataframe=full_df,
    img_dir=train_set_dir,
    target_size=IMG_RESIZE,
    mask_threshold=MASK_THRESHOLD,
    overlap_ratio=0.0,
    normalize=True,
)

NORMALIZATION_MEAN = custom_mean
NORMALIZATION_STD = custom_std

Calculating stats on Training Data...
Computing dataset Mean and Std (this may take a moment)...
Processing 581 slides with Grid Tiling (Thr=0.05, Overlap=0.0)...


  0%|          | 0/581 [00:00<?, ?it/s]

Extraction complete. Total patches: 919


  0%|          | 0/29 [00:00<?, ?it/s]


DONE. Copy these values into your config:
NEW_MEAN = [0.5219733119010925, 0.39480817317962646, 0.5568122267723083]
NEW_STD = [0.10120205581188202, 0.08823584765195847, 0.07768534123897552]


### **Transforms**


In [95]:
# Define transformations

data_transforms = transforms.Compose(
    [
        transforms.Resize(IMG_RESIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=NORMALIZATION_MEAN, std=NORMALIZATION_STD),
    ]
)

## 🧮 **Network Parameters**


### **Custom Nets**


In [96]:
class EfficientNetCustom(nn.Module):
    """
    Instantiates EfficientNet-B0 with ImageNet weights.
    Replaces the classifier head with a high-dropout dense layer to prevent overfitting.
    """

    def __init__(self, num_classes, dropout_rate=0.4):
        super().__init__()
        self.dropout_rate = dropout_rate
        self.num_classes = num_classes

        self.weights = torchvision.models.EfficientNet_V2_S_Weights.DEFAULT
        self.backbone = torchvision.models.efficientnet_v2_s(weights=self.weights)

        in_features = self.backbone.classifier[1].in_features
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(self.dropout_rate),
            nn.Linear(in_features, self.num_classes),  # type: ignore
        )
        self.freeze_backbone()

    def freeze_backbone(self):
        # Freeze all layers except the classifier head
        for name, param in self.backbone.named_parameters():
            if not name.startswith("classifier"):
                param.requires_grad = False
        # Ensure classifier params are trainable
        for param in self.backbone.classifier.parameters():
            param.requires_grad = True

    def unfreeze_backbone(self, n_layers, all=False):
        if all:
            for param in self.backbone.parameters():
                param.requires_grad = True
            return
        # Unfreeze the last n_layers of the backbone (excluding classifier which is already trainable)
        child_counter = 0
        for child in reversed(list(self.backbone.children())):
            child_counter += 1
            if child_counter <= n_layers:
                for param in child.parameters():
                    param.requires_grad = True

    def forward(self, x):
        return self.backbone(x)

    def extract_embeddings(self, x):
        """Returns the flattened feature vector before the classifier."""
        # 1. Run features
        x = self.backbone.features(x)
        # 2. Global Average Pooling (same as original forward)
        x = self.backbone.avgpool(x)
        # 3. Flatten
        x = torch.flatten(x, 1)
        return x

In [97]:
import torch.nn.functional as F


class DenseNetCustom(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.4):
        super().__init__()
        self.dropout_rate = dropout_rate
        self.num_classes = num_classes

        self.weights = torchvision.models.DenseNet121_Weights.DEFAULT
        self.backbone = torchvision.models.densenet121(weights=self.weights)

        # DenseNet classifier is stored in .classifier
        in_features = self.backbone.classifier.in_features

        # Replace Classifier
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(dropout_rate), nn.Linear(in_features, num_classes)
        )

        self.freeze_backbone()

    def freeze_backbone(self):
        # Freeze all feature layers
        for param in self.backbone.features.parameters():
            param.requires_grad = False
        # Unfreeze classifier
        for param in self.backbone.classifier.parameters():
            param.requires_grad = True

    def unfreeze_backbone(self, n_blocks, all=False):
        if all:
            for param in self.backbone.parameters():
                param.requires_grad = True
            return

        # Keep classifier trainable
        for param in self.backbone.classifier.parameters():
            param.requires_grad = True

        # Unfreeze the last n_blocks within features
        children = list(self.backbone.features.children())
        total_children = len(children)
        if n_blocks <= 0:
            return

        start = max(0, total_children - n_blocks)
        for i in range(start, total_children):
            for param in children[i].parameters():
                param.requires_grad = True

    def forward(self, x):
        return self.backbone(x)

    def extract_embeddings(self, x):
        """Returns the flattened feature vector before the classifier."""
        # DenseNet features
        features = self.backbone.features(x)
        # ReLU + Pooling (Standard DenseNet finish)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, 1)
        return out

In [98]:
class CustomNet(nn.Module):
    """
    Wrapper that can interchange between DenseNetCustom and CustomNet.
    Keeps the same constructor signature used in the notebook:
    CustomNet(num_classes, dropout_rate, backbone=...)
    """

    def __init__(self, num_classes, dropout_rate=0.4, backbone="densenet121"):
        super().__init__()
        self.num_classes = num_classes
        self.dropout_rate = dropout_rate
        self.backbone_name = backbone.lower()

        if self.backbone_name in ("densenet", "densenet121"):
            self.backbone = DenseNetCustom(
                num_classes=num_classes, dropout_rate=dropout_rate
            )
        elif self.backbone_name in (
            "efficientnet",
            "efficientnet_v2s",
            "efficientnetv2s",
        ):
            self.backbone = EfficientNetCustom(
                num_classes=num_classes, dropout_rate=dropout_rate
            )
        else:
            raise ValueError(
                f"Unsupported backbone '{backbone}'. Use 'densenet' or 'efficientnet'."
            )

    def freeze_backbone(self):
        # Delegate to underlying implementation
        if hasattr(self.backbone, "freeze_backbone"):
            self.backbone.freeze_backbone()

    def unfreeze_backbone(self, n_layers, all=False):
        # Delegate to underlying implementation
        if hasattr(self.backbone, "unfreeze_backbone"):
            self.backbone.unfreeze_backbone(n_layers, all=all)

    def forward(self, x):
        return self.backbone(x)

    def extract_embeddings(self, x):
        return self.backbone.extract_embeddings(x)

## **XGboost**


In [99]:
def top_k_mean_aggregation(prob_matrix, k_percent=0.3):
    """
    Aggregates patch probabilities into a slide prediction by averaging
    only the most confident patches (Top-K%).

    Args:
        prob_matrix: Numpy array of shape [N_patches, N_classes]
        k_percent: Float (0.0 to 1.0). Percentage of patches to keep.
                   0.3 means we only average the top 30% scores.
    """
    n_patches = prob_matrix.shape[0]

    # Safety check: if slide has very few patches, keep at least 1
    k = max(1, int(n_patches * k_percent))

    # Sort probabilities for each class INDEPENDENTLY (Axis 0 = patches)
    # We want the highest probabilities for Class 0, Class 1, etc.
    sorted_probs = np.sort(prob_matrix, axis=0)

    # Take the top K (the last K elements in the sorted array)
    top_k_probs = sorted_probs[-k:, :]

    # Average them
    slide_score = np.mean(top_k_probs, axis=0)

    return slide_score

In [100]:
def extract_features_from_probs(prob_matrix):
    """
    Turns a (N_patches, 4) probability matrix into a single feature vector (1, N_features).
    """
    features = []

    # 1. Mean Probabilities (Standard Soft Voting) - 4 features
    mean_probs = np.mean(prob_matrix, axis=0)
    features.extend(mean_probs)

    # 2. Max Probabilities (Detect strong tumor signal) - 4 features
    max_probs = np.max(prob_matrix, axis=0)
    features.extend(max_probs)

    # --- NEW INSERTION HERE ---
    # 3. Top-K Means (The "Clean" Signal)
    # Top 30%: Averages the "surest" 1/3 of the slide.
    # Filters out background but keeps the tumor chunks.
    features.extend(top_k_mean_aggregation(prob_matrix, k_percent=0.3))

    # Top 10%: Very aggressive. Focuses only on the absolute peak regions.
    features.extend(top_k_mean_aggregation(prob_matrix, k_percent=0.1))

    # 3. Standard Deviation (Detect tissue heterogeneity) - 4 features
    std_probs = np.std(prob_matrix, axis=0)
    features.extend(std_probs)

    # 4. Percentiles (Robust Max) - 4 features
    p90_probs = np.percentile(prob_matrix, 90, axis=0)
    features.extend(p90_probs)

    return np.array(features)

In [101]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
import torch.nn.functional as F


def generate_hybrid_dataset(full_df):
    print(">>> Generating Hybrid (Embeddings + Prob Stats) Dataset...")

    # We must recreate the exact same split
    skf = StratifiedKFold(n_splits=K_FOLD_MAX_VALUE, shuffle=True, random_state=SEED)

    # Storage for the Stacking Dataset
    X_list = []  # Features
    y_list = []  # True Labels

    for fold_idx, (train_idx, val_idx) in enumerate(
        skf.split(full_df, full_df["label_index"])
    ):
        if fold_idx >= K_FOLD_LIMIT:
            print(f"Reached K_FOLD_LIMIT of {K_FOLD_LIMIT}. Stopping further folds.")
            break

        print(f"Processing OOF for Fold {fold_idx+1}/{min(K_FOLD_MAX_VALUE, K_FOLD_LIMIT)}...")

        fold_val_df = full_df.iloc[val_idx].reset_index(drop=True)

        model_path = f"models/{EXPERIMENT_NAME}_fold{fold_idx+1}_ft_model.pt"
        try:
            # Initialize appropriate model
            model = CustomNet(num_classes, FT_DROPOUT_RATE, backbone=NET_NAME).to(
                device
            )
            model.load_state_dict(torch.load(model_path))
            classifier_head = model.backbone.backbone.classifier
            model.eval()
        except FileNotFoundError:
            print(f"Skipping Fold {fold_idx+1} (Model not found)")
            continue

        # Create Dataset/Loader
        val_ds = MaskedGridTileDataset(
            fold_val_df,
            train_set_dir,
            transforms=data_transforms,
            target_size=IMG_RESIZE,
            mask_threshold=MASK_THRESHOLD,
            overlap_ratio=0.0,
            normalize=True,
        )
        val_loader = make_loader(val_ds, BATCH_SIZE, shuffle=False)

        # Run Inference
        slide_feats = {}  # pid -> list of embeddings
        slide_probs = {}  # pid -> list of probabilities
        slide_labels = {}  # { 'img_name': label_idx }

        with torch.no_grad():
            for inputs, labels, parent_ids in tqdm(val_loader, desc="Predicting", leave=False):
                inputs = inputs.to(device)

                # --- TTA: Prepare Views ---
                inputs_orig = inputs
                inputs_h = torch.flip(inputs, [3])
                inputs_v = torch.flip(inputs, [2])

                # --- 1. Extract Embeddings for all views ---
                # Shape: [Batch, D]
                e1 = model.extract_embeddings(inputs_orig)
                e2 = model.extract_embeddings(inputs_h)
                e3 = model.extract_embeddings(inputs_v)

                # Average the embeddings (Feature Stability)
                avg_embeds = (e1 + e2 + e3) / 3.0

                # --- 2. Compute Probabilities for all views ---
                # We pass the specific embeddings to the classifier, then average the result
                # Note: We use model.backbone.classifier because our custom classes wrap it there
                p1 = F.softmax(classifier_head(e1), dim=1)
                p2 = F.softmax(classifier_head(e2), dim=1)
                p3 = F.softmax(classifier_head(e3), dim=1)

                # Average the probabilities (Prediction Stability)
                avg_probs = (p1 + p2 + p3) / 3.0

                # Move to CPU
                avg_embeds = avg_embeds.cpu().numpy()
                avg_probs = avg_probs.cpu().numpy()
                labels = labels.cpu().numpy()

                for i, pid in enumerate(parent_ids):
                    if pid not in slide_feats:
                        slide_feats[pid] = []
                        slide_probs[pid] = []
                        slide_labels[pid] = labels[i]

                    # Accumulate the Averaged TTA results
                    slide_feats[pid].append(avg_embeds[i])
                    slide_probs[pid].append(avg_probs[i])

        # Feature Engineering per Slide
        for pid in slide_feats.keys():
            # A. Embedding Features (Matrix: N_patches x 1024)
            E_mat = np.array(slide_feats[pid])
            feat_emb_mean = np.mean(E_mat, axis=0)
            feat_emb_max = np.max(E_mat, axis=0)

            # B. Probability Features (Matrix: N_patches x 4)
            P_mat = np.array(slide_probs[pid])

            # Re-use your existing feature extraction logic for probs
            # (Mean, Max, Std, Top-K, etc.)
            feat_prob_stats = extract_features_from_probs(P_mat)

            # C. Concatenate: [Embed_Mean, Embed_Max, Prob_Stats]
            final_vec = np.concatenate([feat_emb_mean, feat_emb_max, feat_prob_stats])

            X_list.append(final_vec)
            y_list.append(slide_labels[pid])

    # Convert to Arrays
    X_stack = np.array(X_list)
    y_stack = np.array(y_list)
    return X_stack, y_stack

In [102]:
X_stack, y_stack = generate_hybrid_dataset(full_df)
print(f"OOF Dataset Created. Shape: {X_stack.shape}")

>>> Generating Hybrid (Embeddings + Prob Stats) Dataset...
Processing OOF for Fold 1/3...
Processing 117 slides with Grid Tiling (Thr=0.01, Overlap=0.0)...


  0%|          | 0/117 [00:00<?, ?it/s]

Extraction complete. Total patches: 461


Predicting:   0%|          | 0/15 [00:00<?, ?it/s]

Processing OOF for Fold 2/3...
Processing 116 slides with Grid Tiling (Thr=0.01, Overlap=0.0)...


...

Processing OOF for Fold 3/3...
Processing 116 slides with Grid Tiling (Thr=0.01, Overlap=0.0)...


  0%|          | 0/116 [00:00<?, ?it/s]

Extraction complete. Total patches: 447


Predicting:   0%|          | 0/14 [00:00<?, ?it/s]

Reached K_FOLD_LIMIT of 3. Stopping further folds.
OOF Dataset Created. Shape: (349, 2072)


In [103]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


def apply_pca(pca_n_components=0.95, X_stack=X_stack, n_embeddings=2048):
    """
    Applies PCA to reduce dimensionality of the hybrid dataset.
    """

    # Split the data
    X_embeds = X_stack[:, :n_embeddings] 
    X_stats = X_stack[:, n_embeddings:]

    # Standardize Embeddings (Required for PCA)
    scaler = StandardScaler()
    X_embeds_scaled = scaler.fit_transform(X_embeds)

    pca = PCA(n_components=pca_n_components, random_state=SEED)
    X_embeds_pca = pca.fit_transform(X_embeds_scaled)

    print(f"PCA reduced embeddings from {X_embeds.shape[1]} to {X_embeds_pca.shape[1]}")

    # Re-concatenate: [Compact Embeddings] + [Original Stats]
    X_stack_optimized = np.concatenate([X_embeds_pca, X_stats], axis=1)

    print(f"New Feature Vector Shape: {X_stack_optimized.shape}")

    return X_stack_optimized, pca, scaler

In [104]:
dummy_probs = np.zeros((10, num_classes))  # 10 dummy patches, 4 classes
dummy_stats_vec = extract_features_from_probs(dummy_probs)
n_stats_features = len(dummy_stats_vec)

# The rest of the columns MUST be the embeddings
n_embed_features = X_stack.shape[1] - n_stats_features
print(f"Number of Embedding Features: {n_embed_features}")

X_stack, pca_model, scaler_model = apply_pca(
    pca_n_components=0.9, X_stack=X_stack, n_embeddings=n_embed_features
)

Number of Embedding Features: 2048
PCA reduced embeddings from 2048 to 125
New Feature Vector Shape: (349, 149)


In [ ]:
from sklearn.utils.class_weight import compute_sample_weight


def grid_search_xgboost(X_stack, y_stack):
    print(">>> Starting XGBoost Hyperparameter Grid Search...")
    max_folds = 3

    param_grid = {
        "n_estimators": [400, 800],
        "max_depth": [5, 7],
        "learning_rate": [0.1, 0.01],
        "colsample_bytree": [0.9],
        "subsample": [0.5],
    }

    best_params = None
    best_score = 0.0
    best_model = None

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

    for params in product(*param_grid.values()):
        param_dict = dict(zip(param_grid.keys(), params))

        fold_scores = []
        fold_models = []

        for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_stack, y_stack)):
            if fold_idx >= max_folds:
                break
            X_train, X_val = X_stack[train_idx], X_stack[val_idx]
            y_train, y_val = y_stack[train_idx], y_stack[val_idx]

            sample_weights_train = compute_sample_weight(
                class_weight="balanced", y=y_train
            )

            model = xgb.XGBClassifier(
                objective="multi:softmax",
                num_class=num_classes,
                random_state=SEED,
                reg_alpha=0.1,
                reg_lambda=1.3,
                early_stopping_rounds=100,
                **param_dict,
            )
            model.fit(
                X_train,
                y_train,
                sample_weight=sample_weights_train,
                eval_set=[(X_val, y_val)],
                verbose=False,
            )
            preds = model.predict(X_val)
            fold_scores.append(f1_score(y_val, preds, average="macro"))
            fold_models.append(model)

        mean_score = np.mean(fold_scores)
        print(f"Params: {param_dict} => Mean F1 Score: {mean_score:.4f}")
        if mean_score > best_score:
            best_score = mean_score
            best_params = param_dict
            best_model = fold_models[int(np.argmax(fold_scores))]

    print(f"Best F1 Score: {best_score:.4f} with params: {best_params}")
    return best_model, best_params

In [110]:
best_model, best_params = grid_search_xgboost(X_stack, y_stack)
print("XGBoost Grid Search Complete. Best Model Obtained.")
print(f"Best Hyperparameters: {best_params}")

>>> Starting XGBoost Hyperparameter Grid Search...
Params: {'n_estimators': 500, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 1.0, 'subsample': 0.5} => Mean F1 Score: 0.3179
Params: {'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytree': 1.0, 'subsample': 0.5} => Mean F1 Score: 0.3530
Best F1 Score: 0.3530 with params: {'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytree': 1.0, 'subsample': 0.5}
XGBoost Grid Search Complete. Best Model Obtained.
Best Hyperparameters: {'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytree': 1.0, 'subsample': 0.5}


In [ ]:
def generate_xgboost_submission(test_loader, cnn_exp_name, xgb_model, scaler, pca, n_embed_features):
    print("--- Running Inference with XGBoost Stacking (Vote of Experts)...")

    # Storage for FINAL XGBoost predictions per slide
    # { 'slide_id': [xgb_prob_fold1, xgb_prob_fold2, ...] }
    slide_final_preds = {}

    # 2. Iterate Folds: Run CNN -> Extract Features -> Predict XGB -> Store
    max_fold = min(K_FOLD_MAX_VALUE, K_FOLD_LIMIT)
    for fold in range(1, max_fold + 1):
        model_path = f"models/{cnn_exp_name}_fold{fold}_ft_model.pt"
        print(f"\n--- Processing Fold {fold}/{max_fold} ---")

        # Load CNN
        try:
            model = CustomNet(num_classes, FT_DROPOUT_RATE, backbone=NET_NAME).to(
                device
            )
            model.load_state_dict(torch.load(model_path))
            classifier_head = model.backbone.backbone.classifier
            model.eval()
        except FileNotFoundError:
            print(f"WARNING: Skipping Fold {fold} (Model not found)")
            continue

        # Dictionary to collect patches for this SPECIFIC fold
        slide_feats = {}  # { 'pid': [ [p0..p3], [p0..p3] ] }
        slide_probs = {}  # pid -> list of probabilities

        with torch.no_grad():
            for inputs, _, parent_ids in tqdm(
                test_loader, desc=f"Inference Fold {fold}", leave=False
            ):
                inputs = inputs.to(device)

                # --- TTA: Prepare Views ---
                inputs_orig = inputs
                inputs_h = torch.flip(inputs, [3])
                inputs_v = torch.flip(inputs, [2])

                # --- 1. Extract Embeddings for all views ---
                # Shape: [Batch, D]
                e1 = model.extract_embeddings(inputs_orig)
                e2 = model.extract_embeddings(inputs_h)
                e3 = model.extract_embeddings(inputs_v)

                # Average the embeddings (Feature Stability)
                avg_embeds = (e1 + e2 + e3) / 3.0

                # --- 2. Compute Probabilities for all views ---
                # We pass the specific embeddings to the classifier, then average the result
                # Note: We use model.backbone.classifier because our custom classes wrap it there
                p1 = F.softmax(classifier_head(e1), dim=1)
                p2 = F.softmax(classifier_head(e2), dim=1)
                p3 = F.softmax(classifier_head(e3), dim=1)

                # Average the probabilities (Prediction Stability)
                avg_probs = (p1 + p2 + p3) / 3.0

                # Move to CPU
                avg_embeds = avg_embeds.cpu().numpy()
                avg_probs = avg_probs.cpu().numpy()

                # Collect patches for this fold
                for i, pid in enumerate(parent_ids):
                    if pid not in slide_feats:
                        slide_feats[pid] = []
                        slide_probs[pid] = []

                    # Accumulate the Averaged TTA results
                    slide_feats[pid].append(avg_embeds[i])
                    slide_probs[pid].append(avg_probs[i])

        # 3. Feature Engineer & Predict with XGBoost for THIS fold
        print(f"XGBoost Prediction for Fold {fold}...")
        for pid in slide_feats.keys():
            # A. Embedding Features (Matrix: N_patches x 1024)
            E_mat = np.array(slide_feats[pid])
            feat_emb_mean = np.mean(E_mat, axis=0)
            feat_emb_max = np.max(E_mat, axis=0)

            # B. Probability Features (Matrix: N_patches x 4)
            P_mat = np.array(slide_probs[pid])

            # Re-use your existing feature extraction logic for probs
            # (Mean, Max, Std, Top-K, etc.)
            feat_prob_stats = extract_features_from_probs(P_mat)

            # C. Concatenate: [Embed_Mean, Embed_Max, Prob_Stats]
            final_vec = np.concatenate([feat_emb_mean, feat_emb_max, feat_prob_stats])

            vec_embed = final_vec[:n_embed_features].reshape(1, -1)
            vec_stats = final_vec[n_embed_features:].reshape(1, -1)

            vec_embed_scaled = scaler.transform(vec_embed)
            vec_embed_pca = pca.transform(vec_embed_scaled)
            
            final_vec_reduced = np.concatenate([vec_embed_pca, vec_stats], axis=1)

            # Predict using XGBoost
            # We use predict_proba to get soft voting capability for the stacker
            # Shape: (1, n_classes)
            xgb_prob = xgb_model.predict_proba(final_vec_reduced)[0]

            if pid not in slide_final_preds:
                slide_final_preds[pid] = []
            slide_final_preds[pid].append(xgb_prob)

    # 4. Average XGBoost Predictions (Soft Voting of Stackers)
    final_rows = []
    print("\nAggregating Ensemble Predictions...")

    for pid, preds_list in slide_final_preds.items():
        # preds_list is a list of arrays (one per fold)
        # Average them
        avg_xgb_probs = np.mean(preds_list, axis=0)

        # Final Argmax
        pred_idx = np.argmax(avg_xgb_probs)
        pred_label = class_names[pred_idx]

        final_rows.append({"sample_index": pid, "label": pred_label})

    # Save
    sub = pd.DataFrame(final_rows).sort_values("sample_index")
    print("Submission DataFrame created")
    return sub

In [ ]:
test_ds = MaskedGridTileDataset(
    dataframe=None,
    img_dir=test_set_dir,
    transforms=data_transforms,
    target_size=IMG_RESIZE,
    mask_threshold=MASK_THRESHOLD,
    overlap_ratio=0.5,
    normalize=True,
)

test_loader = make_loader(test_ds, BATCH_SIZE, shuffle=False)

Processing 477 slides with Grid Tiling (Thr=0.1, Overlap=0.5)...


  0%|          | 0/477 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# EXECUTE
submission_df = generate_xgboost_submission(
    test_loader=test_loader,
    cnn_exp_name=EXPERIMENT_NAME,
    xgb_model=best_model,
    pca=pca_model,
    scaler=scaler_model,
    n_embed_features=n_embed_features,
)

In [ ]:
os.makedirs("submission", exist_ok=True)

submission_df.to_csv(submission_path, index=False)
print(f"✅ Saved Robust Stacking Submission: {submission_path}")

In [ ]:
submission_df